In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/checking-logs.sqlite")

query_test = """
SELECT
    CASE
        WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
        ELSE 'after'
    END AS time,
    AVG((julianday(datetime(dl.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS avg_diff
FROM test_group t
JOIN deadlines dl ON t.labname = dl.labs
WHERE t.labname != 'project1'
GROUP BY time
HAVING COUNT(*) > 1;
"""
test_results = pd.read_sql(query_test, conn)

query_control = """
SELECT
    CASE
        WHEN c.first_commit_ts < c.first_view_ts THEN 'before'
        ELSE 'after'
    END AS time,
    AVG((julianday(datetime(dl.deadlines, 'unixepoch')) - julianday(c.first_commit_ts)) * 24) AS avg_diff
FROM control_group c
JOIN deadlines dl ON c.labname = dl.labs
WHERE c.labname != 'project1'
GROUP BY time
HAVING COUNT(*) > 1;
"""
control_results = pd.read_sql(query_control, conn)

conn.close()

print("Test group results:\n", test_results)
print("Control group results:\n", control_results)


Test group results:
      time    avg_diff
0   after  103.953310
1  before   61.156438
Control group results:
      time    avg_diff
0   after  113.232196
1  before   99.901295
